# Chem

> Chemistry related functions

In [ ]:
#| default_exp chem

In [ ]:
#| hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
from chem_templates.imports import *
from chem_templates.utils import *
import rdkit
from rdkit import Chem

## RDKit i/o

Functions for converting between SMILES strings and RDKit mol objects

In [ ]:
#| export
def to_mol(smile: str) -> Union[Chem.Mol, None]:

    mol = Chem.MolFromSmiles(smile)
    if mol is not None:
        try:
            Chem.SanitizeMol(mol)
        except:
            mol = None
        
    return mol

def to_smile(mol: Chem.Mol) -> str:
    smile = Chem.MolToSmiles(mol)
    return smile

def to_kekule(smile: str) -> str:
    return Chem.MolToSmiles(to_mol(smile), kekuleSmiles=True)

def canon_smile(smile: str) -> str:
    try:
        return Chem.CanonSmiles(smile)
    except:
        return ''
    
def remove_stereo(mol: Chem.Mol) -> Chem.Mol:
    Chem.rdmolops.RemoveStereochemistry(mol)
    return mol

def remove_stereo_smile(smile: str) -> str:
    if '@' in smile:
        mol = to_mol(smile)
        mol = remove_stereo(mol)
        smile = to_smile(mol)
    return smile

In [ ]:
assert type(to_mol('CCC')) == Chem.Mol
assert type(to_smile(Chem.MolFromSmiles('CCC'))) == str

In [ ]:
#| export
class Molecule():
    def __init__(self, smile: str, data: Optional[dict]=None):
        self.smile = canon_smile(smile)
        self.mol = to_mol(self.smile)
        self.valid = (self.mol is not None) and (self.smile != '')
        
        self.data = {}
        self.add_data(data)
            
    def add_data(self, data: Optional[dict]=None):
        if data is not None:
            self.data.update(data)

In [ ]:
#| export
class Reaction():
    def __init__(self, reaction_smarts: str):
        self.reaction_smarts = reaction_smarts
        self.reaction = AllChem.ReactionFromSmarts(self.reaction_smarts)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()